<a href="https://colab.research.google.com/github/meisisoiisme/FH-ML_PyTorch/blob/main/Create%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
pip install GPy

In [52]:
from google.colab import files
from zipfile import ZipFile
import pandas as pd
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
from sklearn.preprocessing import StandardScaler
import GPy
from GPy.models import GPClassification


In [20]:
# Upload the zipfile containing two folders and CSV files
uploaded = files.upload()

# Specify the name of the uploaded zipfile
zipfile_name = next(iter(uploaded))

# Create a directory to extract the contents
extracted_dir = 'extracted_files'
os.makedirs(extracted_dir, exist_ok=True)

# Extract the contents of the zipfile
with ZipFile(zipfile_name, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# List the files in the extracted directory
extracted_files = os.listdir(extracted_dir)
print("Extracted files:", extracted_files)




Saving Data.zip to Data (1).zip
Extracted files: ['Data']


In [65]:
dataset = datasets.ImageFolder(root='/content/extracted_files/Data/images', transform=data_transform, target_transform=lambda x: x)


NameError: name 'datasets' is not defined

In [21]:
# Images - Path
img_path = "/content/extracted_files/Data/images"

# Landmarks - File Adress
rawLandmarks_MetaData = "/content/extracted_files/Data/Landmarks/rawLandmarks_MetaData.csv"
rawLandmarks_X = "/content/extracted_files/Data/Landmarks/rawLandmarks_X.csv"
rawLandmarks_Y = "/content/extracted_files/Data/Landmarks/rawLandmarks_Y.csv"

targetID = "/content/extracted_files/Data/targetID.csv"
labels = "/content/extracted_files/Data/labels.csv"
classes = "/content/extracted_files/Data/classes.csv"
fullfeatures = "/content/extracted_files/Data/FullFeatures.csv"

BGPLVM_data = "/content/extracted_files/Data/DUMMYBGPLVM_DATA.csv"
PROCRUSTES_data = "/content/extracted_files/Data/DUMMYPROCRUSTES_DATA.csv"



In [64]:
# Update the file names accordingly
data_file_name = rawLandmarks_MetaData
feature_selection_name = targetID

# Load feature selection if available
if 'feature_selection_name' in locals():
    feature_selection = np.genfromtxt(feature_selection_name, delimiter=',', dtype=int)
else:
    feature_selection = None

# Load data file
data_file = pd.read_csv(data_file_name, header=None).values

k_fold = 10  # k-fold cross-validation
print("Got path to data: %s" % data_file_name)
print("Path to feature selection: %s" % feature_selection_name)
print("Use %d for k-fold cross-validation" % k_fold)

#-------------------#
#--- Create data ---#
#-------------------#
sample_ID = data_file[:, 0]                        # The sample ID
data_raw = data_file[:, 1:data_file.shape[1]]       # The label followed by data (BGPLVM features)

#------------------#
#--- Do looping ---#
#------------------#
for ITERATION in range(0, 10):  # Assuming 10 iterations
    print("Iteration %d" % ITERATION)
    #--- Create data ---#
    index = np.random.choice(data_raw.shape[0], size=209, replace=False)  # Get sampled random indices
    X_train = data_raw[index, 1:data_raw.shape[1]]  # Get BGPLVM data
    if feature_selection is not None:
        X_train = X_train[:, feature_selection]
    y_train = data_raw[index, 0].astype(int)  # Get labels

    #-----------------------------#
    #--- Do data preprocessing ---#
    #-----------------------------#
    scaler_train = StandardScaler()
    scaler_train.fit(X_train)
    X_train = scaler_train.transform(X_train)

    #-------------------------#
    #--- Do classification ---#
    #-------------------------#
    kernel = GPy.kern.RBF(X_train.shape[1]) + GPy.kern.White(X_train.shape[1], variance=1e-5)
    model = GPy.models.GPClassification(X_train, y_train.reshape(-1, 1), kernel=kernel)

    # Model training
    model.optimize()

    # Get predictions
    predictions = model.predict(X_train)
    predicted_labels = (predictions.mean > 0.5).astype(int)

    # Data iteration
    data_iteration = np.concatenate((sample_ID[index].reshape((209, 1)),  # Sample name
                                     y_train.reshape((209, 1)),  # True class
                                     predicted_labels.reshape((209, 1)),  # Predicted class
                                     predictions.uncertainty().reshape((209, 1))),  # Uncertainty
                                    axis=1)

    prefix = str(ITERATION)
    #--- Create csv file for later processing ---#
    df = pd.DataFrame(data=data_iteration,
                      columns=["sample_ID", "true_class", "predicted_class", "uncertainty"])
    df.to_csv(prefix + "_dataMatrix.csv", index=False)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [45]:
# Import necessary libraries
import datetime
import numpy as np
from sklearn import preprocessing
from GPData import GPData  # Import GPData class

# Specify the paths to your data
path_to_images = '/content/data'  # Replace with the actual path
path_inv_selection = '/content/data/inv_sel'   # Replace with the path to the inverse selection file if applicable
path_faulty = '/content/data/faulty'   # Replace with the path to the faulty elements file if applicable

# Create an instance of the GPData class
dataset = GPData(path_to_images, path_inv_selection, path_faulty, name="myImageDataHandler", scale=True)

# Print information about the loaded data
dataset.Print("Loaded raw data [" + str(dataset.design_Raw.shape[0]) + "x" + str(dataset.design_Raw.shape[1]) + "]")
dataset.Print("Found " + str(len(dataset.index_faulty)) + " faulty elements")
dataset.Print(str(dataset.index_selection_inv))
dataset.Print("Ignore " + str(len(dataset.index_selection_inv)) + " dimensions")


ModuleNotFoundError: No module named 'GPData'

In [ ]:
# Load the CSV file
df = pd.read_csv(landmark_data_path)

class FishLandmarkDataset(Dataset):
    def __init__(self, csv_data, img_dir, transform=None):
        self.landmarks_frame = pd.read_csv(csv_data)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, f"{self.landmarks_frame.iloc[idx, 0]}.jpg")
        image = Image.open(img_name)

        landmarks = self.landmarks_frame.iloc[idx, 2:].to_numpy(dtype=float)
        landmarks = landmarks.reshape(-1, 2)

        if self.transform:
            image = self.transform(image)

        return image, landmarks

# Define transforms
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create dataset
fish_dataset = FishLandmarkDataset(csv_data=landmark_data_path, img_dir=extracted_dir, transform=data_transform)

# Create data loader
fish_dataloader = DataLoader(fish_dataset, batch_size=8, shuffle=True, num_workers=4)

# Iterate through the dataset
for i, (images, landmarks) in enumerate(fish_dataloader):
    # Do something with the batches of images and landmarks
    print(f"Batch {i+1}: Images shape {images.shape}, Landmarks shape {landmarks.shape}")
    if i == 2:  # Stop after 3 batches for example
        break